In [2]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

import config as c

In [5]:
# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

In [6]:
# SQL query
data_sql = """
SELECT *
FROM final_dataset;
"""

# load in tables as dataframes
data_df = pd.read_sql(data_sql, con=connection)

In [ ]:
# Define possible Confusion Matrix Conditions and Assign 'Friendly' Values

conditions = [
  (data_df['HallOfFameStatus'] == 'Not Inducted') & (data_df['ML_Prediction'] == 0),
  (data_df['HallOfFameStatus'] == 'Hall of Fame Member') & (data_df['ML_Prediction'] == 0),
  (data_df['HallOfFameStatus'] == 'Not Inducted') & (data_df['ML_Prediction'] == 1),
  (data_df['HallOfFameStatus'] == 'Hall of Fame Member') & (data_df['ML_Prediction'] == 1)
]

outputs = ['True Negative', 'False Negative', 'False Positive', 'True Positive']

data_df['ConfusionMatrix'] = pd.Series(np.select(conditions, outputs, 'Error'))

data_df.head(50)

In [ ]:
import psycopg2

# connection to database
connection = psycopg2.connect(
    host = c.host,
    port = c.port,
    user = c.user,
    password = c.password,
    database = c.database
    )
cursor=connection.cursor()

rows = zip(data_df.GUID, data_df.ConfusionMatrix)
cur.execute("""CREATE TEMP TABLE "bdb_Draft_test" ("GUID" text, "ConfusionMatrix" text) ON COMMIT DROP""")
cur.executemany("""INSERT INTO "bdb_Draft_test" ("GUID", "ConfusionMatrix") VALUES(%s, %s)""", rows)

cur.execute("""
    ALTER TABLE comprehensive_dataset
    DROP COLUMN IF EXISTS "ConfusionMatrix";
    
    ALTER TABLE comprehensive_dataset
    ADD COLUMN "ConfusionMatrix" text;
   
    UPDATE comprehensive_dataset
    SET "ConfusionMatrix" = "bdb_Draft_test"."ConfusionMatrix"
    FROM "bdb_Draft_test"
    WHERE comprehensive_dataset."GUID" = "bdb_Draft_test"."GUID";
    """)

cur.rowcount
conn.commit()
cur.close()
conn.close()